In [64]:
import re
import time
import traceback
import json
import logging
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, field
from enum import Enum
import os
from openai import OpenAI

# === Configuration Management ===
class Config:
    """Configuration management for Synapse"""
    def __init__(self):
        # Set your actual OpenAI API key here or use environment variable
        self.openai_api_key = os.getenv('OPENAI_API_KEY', 'Inset Your Open API Key Here')
        self.log_level = os.getenv('LOG_LEVEL', 'WARNING')  # Less verbose by default
        self.enable_gpt_summary = os.getenv('ENABLE_GPT_SUMMARY', 'true').lower() == 'true'
        self.confidence_threshold = float(os.getenv('CONFIDENCE_THRESHOLD', '0.7'))
        self.enable_stroke_detection = os.getenv('ENABLE_STROKE_DETECTION', 'true').lower() == 'true'
        self.enable_seizure_detection = os.getenv('ENABLE_SEIZURE_DETECTION', 'true').lower() == 'true'

config = Config()

# === Severity Levels ===
class Severity(Enum):
    CRITICAL = "🚨"
    WARNING = "⚠️"
    MOTOR = "🦴"
    NEURO = "🧠"
    MONITORING = "🔍"
    NORMAL = "✅"

# === Structured Clinical Findings ===
@dataclass
class ClinicalFinding:
    """Structured representation of a clinical finding"""
    message: str
    severity: Severity
    confidence: float
    category: str
    negated: bool = False
    baseline: bool = False
    timestamp: datetime = field(default_factory=datetime.now)

@dataclass
class ClinicalFindings:
    """Collection of all clinical findings"""
    critical_flags: List[ClinicalFinding] = field(default_factory=list)
    warning_flags: List[ClinicalFinding] = field(default_factory=list)
    motor_flags: List[ClinicalFinding] = field(default_factory=list)
    neuro_flags: List[ClinicalFinding] = field(default_factory=list)
    monitoring_flags: List[ClinicalFinding] = field(default_factory=list)
    normal_findings: List[ClinicalFinding] = field(default_factory=list)
    overall_confidence: float = 0.0
    processing_time_ms: int = 0

    def get_all_findings(self) -> List[ClinicalFinding]:
        """Get all findings sorted by severity"""
        all_findings = (self.critical_flags + self.warning_flags +
                       self.motor_flags + self.neuro_flags +
                       self.monitoring_flags + self.normal_findings)
        return sorted(all_findings, key=lambda x: list(Severity).index(x.severity))

# === Compiled Patterns for Performance ===
class CompiledPatterns:
    """Pre-compiled regex patterns for better performance"""
    def __init__(self):
        # Consciousness patterns
        self.gcs_pattern = re.compile(r'gcs\s*(?:of|is|at|score)?\s*(\d+)', re.IGNORECASE)
        self.gcs15_pattern = re.compile(r'gcs\s*(?:of|is|at|score)?\s*15|glasgow\s+(?:coma\s+)?(?:scale|score)\s*(?:of|is|at)?\s*15', re.IGNORECASE)

        # Motor strength patterns - Enhanced for both abbreviated and expanded forms
        self.detailed_strength_pattern = re.compile(r'(rue|lue|rle|lle|right\s+upper\s+extremity|left\s+upper\s+extremity|right\s+lower\s+extremity|left\s+lower\s+extremity)\s+(\d+[-+]?/\d+[-+]?/\d+[-+]?/\d+[-+]?/\d+[-+]?)', re.IGNORECASE)
        self.generalized_strength_pattern = re.compile(r'(rue|lue|rle|lle|right\s+upper\s+extremity|left\s+upper\s+extremity|right\s+lower\s+extremity|left\s+lower\s+extremity)\s+(\d+[-+]?)\s+throughout', re.IGNORECASE)

        # Sensory level pattern - more flexible
        self.sensory_level_pattern = re.compile(r'~?([tl]\d+)\s+sensory\s+level', re.IGNORECASE)

        # Clonus pattern - more flexible
        self.clonus_pattern = re.compile(r'(rt|lt|right|left|r|l)?\s*clonus', re.IGNORECASE)

        # Babinski pattern
        self.babinski_pattern = re.compile(r'(rt|lt|right|left|r|l)?\s*babinski', re.IGNORECASE)

        # Hyperreflexia patterns
        self.hyperreflexia_pattern = re.compile(r'(rue|lue|rle|lle|right|left|bilateral|bl)?\s*(?:dtrs?|reflexes?)\s*(\d+\+)', re.IGNORECASE)

        # Saddle anesthesia pattern
        self.saddle_anesthesia_pattern = re.compile(r'saddle\s+anesthesia', re.IGNORECASE)

        # Sensory deficit patterns
        self.sensory_deficit_pattern = re.compile(r'(?:decreased|diminished|reduced|impaired|absent)\s+sensation', re.IGNORECASE)
        self.bilateral_sensory_pattern = re.compile(r'(?:bilateral|bl|both)\s+(?:feet|soles|lower\s+extremit)', re.IGNORECASE)

        # Reflex patterns
        self.absent_reflexes_pattern = re.compile(r'absent\s+(?:patellar|knee|dtr|reflex)', re.IGNORECASE)
        self.hyporeflexia_pattern = re.compile(r'(?:hyporeflexia|areflexia|absent.*reflex)', re.IGNORECASE)

        # Spinal tumor patterns
        self.spinal_tumor_pattern = re.compile(r'(?:plasmacytoma|myeloma|metastas|tumor).*(?:spine|spinal|vertebr)', re.IGNORECASE)

# Global compiled patterns instance
patterns = CompiledPatterns()

# === Enhanced Abbreviation Dictionary ===
abbreviation_map = {
    # Common medical prefixes
    "pmh": "past medical history",
    "hx": "history",
    "s/p": "status post",
    "p/t": "presenting to",
    "p/w": "presented with",
    "c/f": "concern for",
    "c/b": "complicated by",
    "c/t": "compared to",
    "c/s": "consult",
    "w/": "with",
    "w/o": "without",
    "wo": "without",
    "wwo": "with and without",
    "b/b": "bowel or bladder",
    "2/2": "secondary to",

    # Locations and facilities
    "ED": "emergency department",
    "OSH": "outside hospital",
    "CCH": "Cook County Hospital",
    "NSGY": "neurosurgery",

    # Imaging
    "CTA": "computed tomography angiography",
    "CTH": "CT head",
    "CAP": "chest abdomen pelvis",
    "MRI": "magnetic resonance imaging",
    "XR": "x-ray",

    # Procedures
    "ACDF": "anterior cervical discectomy and fusion",
    "lami": "laminectomy",
    "lamis": "laminectomies",

    # Conditions
    "tSAH": "traumatic subarachnoid hemorrhage",
    "SDH": "subdural hematoma",
    "aSDH": "acute subdural hematoma",
    "mets": "metastases",
    "AMS": "altered mental status",
    "AVN": "avascular necrosis",
    "fx": "fracture",
    "comp fx": "compression fracture",

    # Body parts and directions
    "TP": "transverse process",
    "SP": "spinous process",
    "BL": "bilateral",
    "Rt": "right",
    "Lt": "left",
    "L": "left",
    "R": "right",
    "RUE": "right upper extremity",
    "LUE": "left upper extremity",
    "RLE": "right lower extremity",
    "LLE": "left lower extremity",
    "BUE": "bilateral upper extremities",
    "BLE": "bilateral lower extremities",
    "RUL": "right upper lobe",
    "LBP": "low back pain",

    # Examination terms
    "DTR": "deep tendon reflex",
    "DTRs": "deep tendon reflexes",
    "TTP": "tenderness to palpation",
    "MAES": "moves all extremities spontaneously",
    "maes": "moves all extremities spontaneously",
    "EHL": "extensor hallucis longus",

    # Orientation
    "ox0": "not oriented to person, place, or time",
    "ox1": "oriented to person only",
    "ox2": "oriented to person and place or time",
    "ox3": "oriented to person, place, and time",
    "GCS": "Glasgow Coma Scale",

    # Movement responses
    "loc": "localizes to pain",
    "wd": "withdraws to pain",

    # Trauma
    "GLF": "ground level fall",
    "LOC": "loss of consciousness",
    "BHT": "blunt head trauma",
    "MVC": "motor vehicle collision",
    "MVA": "motor vehicle accident",

    # Patient state
    "ADLs": "activities of daily living",
    "ACAP": "anticoagulant or antiplatelet therapy",
    "PVR": "post-void residual",

    # Anatomical regions
    "T-spine": "thoracic spine",
    "L-spine": "lumbar spine",
    "C-spine": "cervical spine",

    # Time descriptors
    "dx": "diagnosed",
    "x3d": "for 3 days",
    "x2d": "for 2 days",
    "x4d": "for 4 days",
    "x10d": "for 10 days",
    "x1w": "for 1 week",
    "x1mo": "for 1 month",

    # Labs
    "Na": "sodium",
    "CBC": "complete blood count",
    "BMP": "basic metabolic panel",
    "Coags": "coagulation studies",
    "HH": "hemoglobin hematocrit",
    "PLT": "platelets",
    "PT": "prothrombin time",
    "INR": "international normalized ratio",
    "EtOH": "alcohol",

    # Status descriptors
    "neg": "negative",
    "nl": "normal",
    "wnl": "within normal limits",
    "N/V": "nausea and vomiting",
    "HA": "headache",
    "Pt": "patient",
    "pt": "patient",
    "s/s": "signs and symptoms",
    "AFO": "ankle-foot orthosis",
    "↓": "decreased",
    "↑": "increased",

    # Comorbidities
    "HTN": "hypertension",
    "HLD": "hyperlipidemia",
    "HF": "heart failure",
    "PE": "pulmonary embolism",
    "DVT": "deep vein thrombosis",
    "Ca": "cancer",

    # Treatments
    "PT": "physical therapy",
    "Tx": "treatment",
    "ASA": "aspirin",
    "tx": "treatment"
}

# === Enhanced Logging System ===
class ClinicalLogger:
    """Enhanced logging system for clinical processing"""
    def __init__(self):
        self.logger = logging.getLogger('synapse')
        self.logger.setLevel(getattr(logging, config.log_level))

        # Create console handler
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        self.logger.addHandler(handler)

        # Audit trail
        self.audit_trail = []

    def log_clinical_processing(self, input_text: str, findings: ClinicalFindings,
                              abbreviations_expanded: List[str]):
        """Log clinical processing details"""
        audit_entry = {
            'timestamp': datetime.now().isoformat(),
            'input_length': len(input_text),
            'abbreviations_expanded': abbreviations_expanded,
            'findings_count': len(findings.get_all_findings()),
            'processing_time_ms': findings.processing_time_ms,
            'confidence': findings.overall_confidence
        }
        self.audit_trail.append(audit_entry)

        self.logger.info(f"Processed clinical text: {len(findings.get_all_findings())} findings, "
                        f"{findings.processing_time_ms}ms, confidence: {findings.overall_confidence:.2f}")

    def log_error(self, component: str, error_code: str, error_message: str,
                  severity: str = "medium", stack_trace: str = None):
        """Log errors with structured format"""
        error_entry = {
            'timestamp': datetime.now().isoformat(),
            'component': component,
            'error_code': error_code,
            'error_message': error_message,
            'severity': severity,
            'stack_trace': stack_trace
        }
        self.audit_trail.append(error_entry)

        self.logger.error(f"[{component}] {error_code}: {error_message}")
        if stack_trace:
            self.logger.debug(stack_trace)

# Global logger instance
clinical_logger = ClinicalLogger()

# === Enhanced Abbreviation Expansion ===
def expand_abbreviations(blurb: str, dictionary: Dict[str, str]) -> Tuple[str, List[str]]:
    """
    Improved abbreviation expansion with context awareness and performance optimization
    """
    if not blurb:
        return blurb, []

    expanded_terms = set()
    blurb_original = blurb

    try:
        # First pass: complete word matches (with boundaries)
        for abbr, full in dictionary.items():
            # Skip very short abbreviations that might cause issues
            if len(abbr) <= 1:
                continue

            # Use word boundaries for complete word matches
            pattern = r'\b' + re.escape(abbr) + r'\b'
            matches = re.findall(pattern, blurb, re.IGNORECASE)

            if matches:
                expanded_terms.add(abbr)
                blurb = re.sub(pattern, full, blurb, flags=re.IGNORECASE)

        # Second pass: special patterns without boundaries
        special_patterns = {
            r's/p': 'status post',
            r'c/f': 'concern for',
            r'p/t': 'presenting to',
            r'c/t': 'compared to',
            r'c/b': 'complicated by',
            r'c/s': 'consult',
            r'w/': 'with',
            r'w/o': 'without',
            r'b/b': 'bowel or bladder',
            r'2/2': 'secondary to',
            r'p/w': 'presented with'
        }

        for pattern, replacement in special_patterns.items():
            matches = re.findall(re.escape(pattern), blurb, re.IGNORECASE)
            if matches:
                expanded_terms.add(pattern)
                blurb = re.sub(re.escape(pattern), replacement, blurb, flags=re.IGNORECASE)

        clinical_logger.logger.debug(f"Expanded {len(expanded_terms)} abbreviations: {', '.join(expanded_terms)}")
        return blurb, list(expanded_terms)

    except Exception as e:
        error_msg = f"Error in abbreviation expansion: {str(e)}"
        clinical_logger.log_error(
            component="abbreviation_expansion",
            error_code="EXPANSION_ERROR",
            error_message=error_msg,
            stack_trace=traceback.format_exc()
        )
        return blurb_original, []

# === Enhanced Negation Detection ===
def is_negated(term: str, blurb: str) -> bool:
    """
    Enhanced negation detection with improved context awareness
    """
    if not term or not blurb:
        return False

    term = term.strip().lower()
    blurb_lower = blurb.lower()

    # Special handling for exam findings - if something appears after "exam:" or "intact except", it's usually a positive finding
    exam_sections = re.split(r'exam\s*:', blurb_lower)
    if len(exam_sections) > 1:
        exam_part = exam_sections[-1]  # Get the part after "exam:"
        if term in exam_part:
            # Check if it's in an "intact except" context - these are positive findings
            if re.search(r'intact\s+except.*?' + re.escape(term), exam_part):
                return False
            # Check if it's directly after exam findings without negation
            if not re.search(r'(?:denies|no|negative|without|absent)\s+.*?' + re.escape(term), exam_part):
                return False

    # Parse the text into sentences to restrict negation scope
    sentences = re.split(r'[.!?]\s+', blurb_lower)

    # Check sentences containing the term
    for sentence in sentences:
        if term in sentence:
            # If the sentence contains "intact except", findings after that are positive
            if 'intact except' in sentence:
                intact_except_pos = sentence.find('intact except')
                term_pos = sentence.find(term)
                if term_pos > intact_except_pos:
                    return False  # Term appears after "intact except" so it's a positive finding

            # Check for denial patterns in the sentence
            denial_patterns = [
                r'denies', r'denied', r'no', r'not', r'negative',
                r'absence of', r'without', r'hasn\'t', r'doesn\'t',
                r'has not', r'does not', r'not (?:having|experiencing|showing)',
                r'hasn\'t endorsed', r'hasn\'t had', r'hasn\'t experienced'
            ]

            # Only consider negation if the denial word is close to the term (within 10 words)
            for pattern in denial_patterns:
                denial_matches = list(re.finditer(pattern, sentence))
                for denial_match in denial_matches:
                    denial_pos = denial_match.end()
                    term_pos = sentence.find(term)
                    if term_pos > denial_pos:
                        # Check if there are too many words between denial and term
                        words_between = len(sentence[denial_pos:term_pos].split())
                        if words_between <= 10:  # Only negate if close proximity
                            # Check for double negatives
                            if not re.search(r'not.+?(?:denied|negative|absent)', sentence):
                                return True

    return False

# === Enhanced Exam Components Extraction ===
def extract_enhanced_exam_components(blurb: str, blurb_lower: str) -> List[ClinicalFinding]:
    """
    Comprehensive exam component extraction with all clinical patterns
    """
    findings = []

    try:
        # Identify exam section
        exam_section = blurb_lower
        exam_section_patterns = [
            r'exam(?:\s*[-:]\s*)(.*?)(?:\n\n|\Z)',
            r'(?:physical|neuro(?:logical)?|motor)\s+exam(?:\s*[-:]\s*)(.*?)(?:\n\n|\Z)',
            r'(?:physical|neurological) findings(?:\s*[-:]\s*)(.*?)(?:\n\n|\Z)'
        ]

        for pattern in exam_section_patterns:
            match = re.search(pattern, blurb_lower, re.DOTALL)
            if match:
                exam_section = match.group(1).strip()
                break

        # === CONSCIOUSNESS AND ORIENTATION ASSESSMENT ===
        # Check for GCS score
        gcs_pattern = r'gcs\s*(?:of|is|at|score)?\s*(\d+)'
        gcs_match = re.search(gcs_pattern, exam_section)

        if gcs_match:
            gcs_score = int(gcs_match.group(1))
            if gcs_score == 15:
                findings.append(ClinicalFinding(
                    message="Full consciousness: GCS 15",
                    severity=Severity.NORMAL,
                    confidence=0.95,
                    category="consciousness"
                ))
            elif gcs_score >= 13:
                findings.append(ClinicalFinding(
                    message=f"Mild consciousness impairment: GCS {gcs_score}",
                    severity=Severity.MONITORING,
                    confidence=0.9,
                    category="consciousness"
                ))
            elif gcs_score >= 9:
                findings.append(ClinicalFinding(
                    message=f"Moderate consciousness impairment: GCS {gcs_score}",
                    severity=Severity.WARNING,
                    confidence=0.95,
                    category="consciousness"
                ))
            else:
                findings.append(ClinicalFinding(
                    message=f"Severe consciousness impairment: GCS {gcs_score}",
                    severity=Severity.CRITICAL,
                    confidence=0.98,
                    category="consciousness"
                ))

        # Check for arousal descriptions
        arousal_patterns = {
            r'(?:eyes?\s+open(?:ing)?)\s+(?:to|with)\s+(?:heavy\s+)?(?:stim|stimulation|pain)': (
                "Decreased arousal level—requires stimulation for eye opening", Severity.NEURO, 0.85),
            r'alert|awake|wide awake': (
                "Alert and awake", Severity.NORMAL, 0.9),
            r'drowsy|lethargic|somnolent': (
                "Decreased arousal—drowsy/lethargic", Severity.NEURO, 0.8)
        }

        for pattern, (message, severity, confidence) in arousal_patterns.items():
            match = re.search(pattern, exam_section)
            if match and not is_negated(match.group(0), blurb_lower):
                findings.append(ClinicalFinding(
                    message=message,
                    severity=severity,
                    confidence=confidence,
                    category="arousal"
                ))

        # Check for orientation status
        orientation_patterns = {
            r'\box0\b|not oriented|disoriented': (
                "Altered mental status: Ox0 or disoriented", Severity.NEURO, 0.9),
            r'\box1\b|oriented to person only': (
                "Partial orientation: Ox1", Severity.NEURO, 0.85),
            r'\box2\b|oriented to person and (place|time)': (
                "Oriented to person and place: Ox2", Severity.MONITORING, 0.8),
            r'\box3\b|oriented x3|fully oriented': (
                "Fully oriented: Ox3", Severity.NORMAL, 0.9)
        }

        for pattern, (message, severity, confidence) in orientation_patterns.items():
            match = re.search(pattern, exam_section)
            if match and not is_negated(match.group(0), blurb_lower):
                findings.append(ClinicalFinding(
                    message=message,
                    severity=severity,
                    confidence=confidence,
                    category="orientation"
                ))
                break  # Only add the most specific orientation finding

        # === MOTOR RESPONSES TO PAINFUL STIMULI ===
        motor_response_patterns = {
            r'bue\s+loc': (
                "Localizes to pain in bilateral upper extremities", Severity.NEURO, 0.85),
            r'ble\s+wd': (
                "Withdrawal response in bilateral lower extremities", Severity.NEURO, 0.85),
            r'bue\s+loc\s+ble\s+wd': (
                "Localizes in upper extremities, withdraws in lower extremities", Severity.NEURO, 0.9),
            r'(?:does not follow|unable to follow) commands': (
                "Decreased level of consciousness—unable to follow commands", Severity.NEURO, 0.9),
            r'follows? commands': (
                "Follows commands appropriately", Severity.NORMAL, 0.85)
        }

        for pattern, (message, severity, confidence) in motor_response_patterns.items():
            match = re.search(pattern, exam_section)
            if match and not is_negated(match.group(0), blurb_lower):
                findings.append(ClinicalFinding(
                    message=message,
                    severity=severity,
                    confidence=confidence,
                    category="motor_response"
                ))

        # === CRANIAL NERVE EXAMINATION ===
        cranial_nerve_patterns = {
            r'(?:facial|face)\s+(?:droop|weakness|asymmetry)': (
                "Facial droop—possible cranial nerve VII weakness", Severity.MOTOR, 0.85),
            r'(?:tongue|lingual)\s+(?:deviation|weakness)': (
                "Tongue deviation—possible cranial nerve XII weakness", Severity.MOTOR, 0.85),
            r'(?:slurred|dysarthric)\s+speech': (
                "Dysarthria/slurred speech—monitor for progression", Severity.NEURO, 0.8)
        }

        for pattern, (message, severity, confidence) in cranial_nerve_patterns.items():
            match = re.search(pattern, exam_section)
            if match and not is_negated(match.group(0), blurb_lower):
                findings.append(ClinicalFinding(
                    message=message,
                    severity=severity,
                    confidence=confidence,
                    category="cranial_nerves"
                ))

        # === COMPREHENSIVE MOTOR STRENGTH ASSESSMENT ===
        limb_map = {
            "rue": "RIGHT UPPER EXTREMITY",
            "lue": "LEFT UPPER EXTREMITY",
            "rle": "RIGHT LOWER EXTREMITY",
            "lle": "LEFT LOWER EXTREMITY",
            "bue": "BILATERAL UPPER EXTREMITIES",
            "ble": "BILATERAL LOWER EXTREMITIES",
            "right arm": "RIGHT UPPER EXTREMITY",
            "left arm": "LEFT UPPER EXTREMITY",
            "right upper extremity": "RIGHT UPPER EXTREMITY",
            "left upper extremity": "LEFT UPPER EXTREMITY",
            "right leg": "RIGHT LOWER EXTREMITY",
            "left leg": "LEFT LOWER EXTREMITY",
            "right lower extremity": "RIGHT LOWER EXTREMITY",
            "left lower extremity": "LEFT LOWER EXTREMITY"
        }

        muscle_groups = {
            "RIGHT UPPER EXTREMITY": ["shoulder abduction", "elbow flexion", "elbow extension", "wrist extension", "handgrip"],
            "LEFT UPPER EXTREMITY": ["shoulder abduction", "elbow flexion", "elbow extension", "wrist extension", "handgrip"],
            "RIGHT LOWER EXTREMITY": ["hip flexion", "knee extension", "ankle dorsiflexion", "EHL", "ankle plantarflexion"],
            "LEFT LOWER EXTREMITY": ["hip flexion", "knee extension", "ankle dorsiflexion", "EHL", "ankle plantarflexion"]
        }

        processed_extremities = set()
        exam_normalized = exam_section

        # Normalize strength notation
        exam_normalized = re.sub(r'(\d+[-+]?)\s+(\d+[-+]?)', r'\1/\2', exam_normalized)
        exam_normalized = re.sub(r'(\d+[-+]?)[,;](\d+[-+]?)', r'\1/\2', exam_normalized)
        exam_normalized = re.sub(r'(\d+)s', r'\1', exam_normalized)

        # Complex multiple muscle strength (e.g., RUE 5/5/5/4-/4)
        complex_pattern = r'(rue|lue|rle|lle|right\s+upper\s+extremity|left\s+upper\s+extremity|right\s+lower\s+extremity|left\s+lower\s+extremity)(?:\s*[:;-])?\s*(?:\()?(\d+[-+]?(?:[\/\s,.-]?\d+[-+]?){2,4})(?:\))?'
        complex_matches = re.finditer(complex_pattern, exam_normalized, re.IGNORECASE)

        for match in complex_matches:
            extremity = match.group(1).lower().replace(' ', ' ')
            if extremity in processed_extremities:
                continue

            extremity_name = limb_map.get(extremity, extremity.upper())
            strength_text = match.group(2)
            values = re.findall(r'(\d+[-+]?)', strength_text)

            weak_values = []
            for i, val in enumerate(values):
                if val.strip() not in ['5', '5+']:
                    if extremity_name in muscle_groups and i < len(muscle_groups[extremity_name]):
                        muscle_name = muscle_groups[extremity_name][i]
                    else:
                        muscle_name = f"muscle {i+1}"
                    weak_values.append(f"{muscle_name}: {val}")

            if weak_values:
                processed_extremities.add(extremity)
                weakness_details = ", ".join(weak_values)

                # Check severity
                has_severe_weakness = any(int(val.strip().replace('+', '').replace('-', '')) <= 2
                                        for val in values if val.strip().replace('+', '').replace('-', '').isdigit())
                severity = Severity.WARNING if has_severe_weakness else Severity.MOTOR
                confidence = 0.95 if has_severe_weakness else 0.85

                findings.append(ClinicalFinding(
                    message=f"Weakness in {extremity_name}: {weakness_details}",
                    severity=severity,
                    confidence=confidence,
                    category="motor"
                ))

        # Simple strength scores (e.g., LUE 4+/5)
        simple_pattern = r'(rue|lue|rle|lle|right arm|left arm|right leg|left leg)\s+(\d+[-+]?)/5'
        simple_matches = re.finditer(simple_pattern, exam_section, re.IGNORECASE)

        for match in simple_matches:
            extremity = match.group(1).lower()
            if extremity in processed_extremities:
                continue

            strength = match.group(2)
            if strength not in ['5', '5+']:
                processed_extremities.add(extremity)
                extremity_name = limb_map.get(extremity, extremity.upper())

                severity = Severity.WARNING if int(strength.replace('+', '').replace('-', '')) <= 2 else Severity.MOTOR
                confidence = 0.9

                findings.append(ClinicalFinding(
                    message=f"Weakness in {extremity_name}: {strength}/5",
                    severity=severity,
                    confidence=confidence,
                    category="motor"
                ))

        # === REFLEX ASSESSMENT ===
        reflex_patterns = {
            r'hoffmans?\b|hoffman\'?s?\s+sign': (
                "Hoffman's sign—UMN risk", Severity.WARNING, 0.85),
            r'babinski\b|babinski\'?s?\s+sign': (
                "Babinski reflex noted", Severity.WARNING, 0.85),
            r'(?:hyperreflexia|[\d][\+]?\s+dtrs?|dtrs?\s+[\d][\+]?)': (
                "Hyperreflexia—monitor for UMN lesion", Severity.WARNING, 0.8),
            r'(?:hyporeflexia|areflexia|0\s+dtrs?|dtrs?\s+0)': (
                "Hyporeflexia—possible LMN involvement", Severity.WARNING, 0.8),
            r'(?:normal\s+reflexes|2\+?\s+dtrs?|dtrs?\s+2\+?)': (
                "Reflexes within normal limits", Severity.NORMAL, 0.9)
        }

        for pattern, (message, severity, confidence) in reflex_patterns.items():
            matches = re.finditer(pattern, exam_section, re.IGNORECASE)
            for match in matches:
                if not is_negated(match.group(0), blurb_lower):
                    findings.append(ClinicalFinding(
                        message=message,
                        severity=severity,
                        confidence=confidence,
                        category="reflexes"
                    ))
                    break

        # === SENSORY ASSESSMENT ===
        sensory_patterns = {
            r'(?:decreased|diminished|reduced|impaired)\s+sensation': (
                "Sensory deficit present", Severity.NEURO, 0.8),
            r'numbness\b|paresthesia\b|dysesthesia\b': (
                "Sensory deficit present", Severity.NEURO, 0.8),
            r'sensation\s+(?:is\s+)?(?:intact|normal|preserved)|silt': (
                "Sensation intact", Severity.NORMAL, 0.9)
        }

        for pattern, (message, severity, confidence) in sensory_patterns.items():
            match = re.search(pattern, exam_section, re.IGNORECASE)
            if match and not is_negated(match.group(0), blurb_lower):
                if "deficit" in message and is_baseline_deficit_mentioned(blurb_lower):
                    findings.append(ClinicalFinding(
                        message="Sensory findings noted but consistent with baseline",
                        severity=Severity.NORMAL,
                        confidence=0.7,
                        category="sensory"
                    ))
                else:
                    findings.append(ClinicalFinding(
                        message=message,
                        severity=severity,
                        confidence=confidence,
                        category="sensory"
                    ))
                break

        # === CAUDA EQUINA ASSESSMENT ===
        # Check for bowel/bladder dysfunction
        bowel_bladder_pattern = r'(?:bowel|bladder)\s+(?:incontinence|retention|dysfunction)'
        bowel_bladder_match = re.search(bowel_bladder_pattern, blurb_lower, re.IGNORECASE)
        if bowel_bladder_match and not is_negated(bowel_bladder_match.group(0), blurb_lower):
            findings.append(ClinicalFinding(
                message="Bowel/bladder dysfunction—evaluate for cauda equina",
                severity=Severity.WARNING,
                confidence=0.9,
                category="cauda_equina"
            ))

        # Check for rectal tone
        rectal_tone_pattern = r'\+rectal\s+tone|normal\s+rectal\s+tone|rectal\s+tone\s+(?:present|intact)'
        rectal_tone_match = re.search(rectal_tone_pattern, exam_section, re.IGNORECASE)
        if rectal_tone_match and not is_negated(rectal_tone_match.group(0), blurb_lower):
            findings.append(ClinicalFinding(
                message="Rectal tone intact",
                severity=Severity.NORMAL,
                confidence=0.9,
                category="cauda_equina"
            ))

        return findings

    except Exception as e:
        clinical_logger.log_error(
            component="extract_enhanced_exam_components",
            error_code="ENHANCED_EXAM_ERROR",
            error_message=str(e),
            stack_trace=traceback.format_exc()
        )
        return []
def risk_flag(blurb: str, prior_findings: Optional[ClinicalFindings] = None) -> ClinicalFindings:
    """
    Enhanced universal risk flagging engine with structured output
    """
    if not blurb:
        return ClinicalFindings(
            critical_flags=[ClinicalFinding(
                message="Empty input—manual review advised",
                severity=Severity.WARNING,
                confidence=0.0,
                category="system"
            )]
        )

    start_time = time.time()
    findings = ClinicalFindings()

    try:
        # Expand abbreviations
        expanded_blurb, expanded_terms = expand_abbreviations(blurb, abbreviation_map)
        blurb_lower = expanded_blurb.lower()

        # === MOTOR STRENGTH ASSESSMENT ===
        limb_map = {
            "rue": "RIGHT UPPER EXTREMITY",
            "lue": "LEFT UPPER EXTREMITY",
            "rle": "RIGHT LOWER EXTREMITY",
            "lle": "LEFT LOWER EXTREMITY",
            "right upper extremity": "RIGHT UPPER EXTREMITY",
            "left upper extremity": "LEFT UPPER EXTREMITY",
            "right lower extremity": "RIGHT LOWER EXTREMITY",
            "left lower extremity": "LEFT LOWER EXTREMITY",
            "right arm": "RIGHT UPPER EXTREMITY",
            "left arm": "LEFT UPPER EXTREMITY",
            "right leg": "RIGHT LOWER EXTREMITY",
            "left leg": "LEFT LOWER EXTREMITY"
        }

        # Detailed strength patterns (e.g., LLE 2/5/4+/4+/5 OR left lower extremity 2/5/4+/4+/5)
        detailed_matches = patterns.detailed_strength_pattern.finditer(expanded_blurb)
        for match in detailed_matches:
            extremity = match.group(1).lower()
            extremity_name = limb_map.get(extremity, extremity.upper())
            strength_text = match.group(2)
            values = re.findall(r'(\d+[-+]?)', strength_text)

            # Analyze the strength values - any value < 5 indicates weakness
            weak_values = []
            muscle_groups = ["hip flexion", "knee extension", "ankle dorsiflexion", "EHL", "ankle plantarflexion"]

            for i, val in enumerate(values):
                val_clean = val.strip().replace('+', '').replace('-', '')
                if val_clean.isdigit() and int(val_clean) < 5:
                    muscle_name = muscle_groups[i] if i < len(muscle_groups) else f"muscle {i+1}"
                    weak_values.append(f"{muscle_name}: {val}")

            if weak_values:
                weakness_details = ", ".join(weak_values)
                # Flag severe weakness (any muscle 2/5 or less) as WARNING instead of just MOTOR
                has_severe_weakness = any(int(val.strip().replace('+', '').replace('-', '')) <= 2
                                        for val in values if val.strip().replace('+', '').replace('-', '').isdigit())
                severity = Severity.WARNING if has_severe_weakness else Severity.MOTOR
                confidence = 0.95 if has_severe_weakness else 0.85

                if severity == Severity.WARNING:
                    findings.warning_flags.append(ClinicalFinding(
                        message=f"Significant weakness in {extremity_name}: {weakness_details}",
                        severity=severity,
                        confidence=confidence,
                        category="motor"
                    ))
                else:
                    findings.motor_flags.append(ClinicalFinding(
                        message=f"Weakness in {extremity_name}: {weakness_details}",
                        severity=severity,
                        confidence=confidence,
                        category="motor"
                    ))

        # Generalized strength patterns (e.g., "LLE 4+ throughout")
        generalized_matches = patterns.generalized_strength_pattern.finditer(expanded_blurb)
        for match in generalized_matches:
            extremity = match.group(1).lower()
            extremity_name = limb_map.get(extremity, extremity.upper())
            strength_value = match.group(2)

            # Convert strength value to numeric for comparison
            strength_numeric = strength_value.replace('+', '').replace('-', '')
            if strength_numeric.isdigit() and int(strength_numeric) < 5:
                severity = Severity.WARNING if int(strength_numeric) <= 3 else Severity.MOTOR
                confidence = 0.9

                if severity == Severity.WARNING:
                    findings.warning_flags.append(ClinicalFinding(
                        message=f"Generalized weakness in {extremity_name}: {strength_value} throughout",
                        severity=severity,
                        confidence=confidence,
                        category="motor"
                    ))
                else:
                    findings.motor_flags.append(ClinicalFinding(
                        message=f"Weakness in {extremity_name}: {strength_value} throughout",
                        severity=severity,
                        confidence=confidence,
                        category="motor"
                    ))

        # === DIRECT SENSORY LEVEL DETECTION ===
        sensory_level_match = patterns.sensory_level_pattern.search(blurb_lower)
        if sensory_level_match and not is_negated(sensory_level_match.group(0), blurb_lower):
            level = sensory_level_match.group(1).upper()
            findings.warning_flags.append(ClinicalFinding(
                message=f"Sensory level at {level}—possible cord involvement",
                severity=Severity.WARNING,
                confidence=0.9,
                category="sensory"
            ))

        # === DIRECT CLONUS DETECTION ===
        clonus_match = patterns.clonus_pattern.search(blurb_lower)
        if clonus_match and not is_negated(clonus_match.group(0), blurb_lower):
            side = clonus_match.group(1) if clonus_match.group(1) else ""
            side_text = f" {side}" if side else ""
            findings.warning_flags.append(ClinicalFinding(
                message=f"Clonus detected{side_text}—UMN involvement",
                severity=Severity.WARNING,
                confidence=0.9,
                category="reflexes"
            ))

        # === BABINSKI DETECTION ===
        babinski_match = patterns.babinski_pattern.search(blurb_lower)
        if babinski_match and not is_negated(babinski_match.group(0), blurb_lower):
            side = babinski_match.group(1) if babinski_match.group(1) else ""
            side_text = f" {side}" if side else ""
            findings.warning_flags.append(ClinicalFinding(
                message=f"Babinski sign{side_text}—UMN involvement",
                severity=Severity.WARNING,
                confidence=0.9,
                category="reflexes"
            ))

        # === HYPERREFLEXIA DETECTION ===
        hyperreflexia_match = patterns.hyperreflexia_pattern.search(blurb_lower)
        if hyperreflexia_match and not is_negated(hyperreflexia_match.group(0), blurb_lower):
            extremity = hyperreflexia_match.group(1) if hyperreflexia_match.group(1) else ""
            reflex_grade = hyperreflexia_match.group(2)
            extremity_text = f" {extremity}" if extremity else ""
            findings.warning_flags.append(ClinicalFinding(
                message=f"Hyperreflexia{extremity_text} ({reflex_grade})—UMN involvement",
                severity=Severity.WARNING,
                confidence=0.85,
                category="reflexes"
            ))

        # === SADDLE ANESTHESIA DETECTION ===
        saddle_match = patterns.saddle_anesthesia_pattern.search(blurb_lower)
        if saddle_match and not is_negated(saddle_match.group(0), blurb_lower):
            findings.critical_flags.append(ClinicalFinding(
                message="Saddle anesthesia—urgent cauda equina evaluation needed",
                severity=Severity.CRITICAL,
                confidence=0.95,
                category="cauda_equina"
            ))

        # === SENSORY DEFICIT DETECTION ===
        sensory_deficit_match = patterns.sensory_deficit_pattern.search(blurb_lower)
        if sensory_deficit_match and not is_negated(sensory_deficit_match.group(0), blurb_lower):
            # Check if it's bilateral
            bilateral_match = patterns.bilateral_sensory_pattern.search(blurb_lower)
            if bilateral_match:
                findings.warning_flags.append(ClinicalFinding(
                    message="Bilateral sensory deficits—possible cord/cauda equina involvement",
                    severity=Severity.WARNING,
                    confidence=0.85,
                    category="sensory"
                ))
            else:
                findings.neuro_flags.append(ClinicalFinding(
                    message="Sensory deficit noted",
                    severity=Severity.NEURO,
                    confidence=0.8,
                    category="sensory"
                ))

        # === REFLEX ABNORMALITIES ===
        absent_reflexes_match = patterns.absent_reflexes_pattern.search(blurb_lower)
        if absent_reflexes_match and not is_negated(absent_reflexes_match.group(0), blurb_lower):
            findings.warning_flags.append(ClinicalFinding(
                message="Absent reflexes—possible lower motor neuron involvement",
                severity=Severity.WARNING,
                confidence=0.9,
                category="reflexes"
            ))

        # === ENHANCED PHYSICAL EXAM ANALYSIS ===
        exam_findings = extract_enhanced_exam_components(expanded_blurb, blurb_lower)
        for finding in exam_findings:
            if finding.severity == Severity.CRITICAL:
                findings.critical_flags.append(finding)
            elif finding.severity == Severity.WARNING:
                findings.warning_flags.append(finding)
            elif finding.severity == Severity.MOTOR:
                findings.motor_flags.append(finding)
            elif finding.severity == Severity.NEURO:
                findings.neuro_flags.append(finding)
            elif finding.severity == Severity.MONITORING:
                findings.monitoring_flags.append(finding)
            else:
                findings.normal_findings.append(finding)

        # === SPINAL TUBERCULOSIS CONTEXT ===
        if re.search(r'potts\s+disease|spinal\s+tb|tuberculosis.*spine', blurb_lower):
            findings.monitoring_flags.append(ClinicalFinding(
                message="Known spinal TB/Potts disease—monitor for neurological progression",
                severity=Severity.MONITORING,
                confidence=0.9,
                category="infection"
            ))

        # === CALCULATE OVERALL CONFIDENCE ===
        all_findings = findings.get_all_findings()
        if all_findings:
            findings.overall_confidence = sum(f.confidence for f in all_findings) / len(all_findings)
        else:
            findings.overall_confidence = 0.0

        # === DEDUPLICATION ===
        def deduplicate_findings(finding_list):
            """Remove duplicate findings based on clinical concepts"""
            unique_findings = []
            seen_concepts = set()

            for finding in finding_list:
                msg_lower = finding.message.lower()

                # Define concept keys for similar findings
                concept_key = None
                if 'babinski' in msg_lower:
                    concept_key = 'babinski'
                elif 'clonus' in msg_lower:
                    concept_key = 'clonus'
                elif 'sensory level' in msg_lower:
                    concept_key = 'sensory_level'
                elif 'saddle anesthesia' in msg_lower:
                    concept_key = 'saddle_anesthesia'
                elif 'hoffman' in msg_lower:
                    concept_key = 'hoffman'
                elif 'hyperreflexia' in msg_lower:
                    concept_key = 'hyperreflexia'
                elif 'weakness' in msg_lower and 'right upper extremity' in msg_lower:
                    concept_key = 'rue_weakness'
                elif 'weakness' in msg_lower and 'left upper extremity' in msg_lower:
                    concept_key = 'lue_weakness'
                elif 'weakness' in msg_lower and 'right lower extremity' in msg_lower:
                    concept_key = 'rle_weakness'
                elif 'weakness' in msg_lower and 'left lower extremity' in msg_lower:
                    concept_key = 'lle_weakness'
                else:
                    # For unique findings, use the full message
                    concept_key = msg_lower.strip()

                # Only add if we haven't seen this concept, or if this one is better
                if concept_key not in seen_concepts:
                    unique_findings.append(finding)
                    seen_concepts.add(concept_key)
                else:
                    # If duplicate, keep the one with higher confidence or more specific message
                    existing_idx = next(i for i, f in enumerate(unique_findings)
                                      if concept_key == 'babinski' and 'babinski' in f.message.lower() or
                                         concept_key == 'clonus' and 'clonus' in f.message.lower() or
                                         concept_key == 'hyperreflexia' and 'hyperreflexia' in f.message.lower() or
                                         concept_key in f.message.lower())

                    if 0 <= existing_idx < len(unique_findings):
                        existing_finding = unique_findings[existing_idx]
                        # Keep the more specific (longer) or higher confidence finding
                        if (finding.confidence > existing_finding.confidence or
                            len(finding.message) > len(existing_finding.message)):
                            unique_findings[existing_idx] = finding

            return unique_findings

        # Apply deduplication to all finding lists
        findings.critical_flags = deduplicate_findings(findings.critical_flags)
        findings.warning_flags = deduplicate_findings(findings.warning_flags)
        findings.motor_flags = deduplicate_findings(findings.motor_flags)
        findings.neuro_flags = deduplicate_findings(findings.neuro_flags)
        findings.monitoring_flags = deduplicate_findings(findings.monitoring_flags)
        findings.normal_findings = deduplicate_findings(findings.normal_findings)

        # Recalculate confidence after deduplication
        all_findings = findings.get_all_findings()
        if all_findings:
            findings.overall_confidence = sum(f.confidence for f in all_findings) / len(all_findings)
        else:
            findings.overall_confidence = 0.0

        # Calculate processing time
        findings.processing_time_ms = int((time.time() - start_time) * 1000)

        # Log processing
        clinical_logger.log_clinical_processing(blurb, findings, expanded_terms)

        return findings

    except Exception as e:
        error_msg = f"Error processing consult text: {str(e)}"
        clinical_logger.log_error(
            component="risk_flag_engine",
            error_code="PROCESSING_ERROR",
            error_message=error_msg,
            severity="high",
            stack_trace=traceback.format_exc()
        )

        return ClinicalFindings(
            critical_flags=[ClinicalFinding(
                message=f"Error in processing: {error_msg}",
                severity=Severity.CRITICAL,
                confidence=0.0,
                category="system"
            )]
        )

# === OpenAI Integration ===
def initialize_openai_client():
    """Initialize OpenAI client with error handling"""
    try:
        return OpenAI(api_key=config.openai_api_key)
    except Exception as e:
        clinical_logger.log_error(
            component="openai_integration",
            error_code="CLIENT_INIT_ERROR",
            error_message=str(e),
            severity="medium"
        )
        return None

# === GPT Summary Integration ===
def gpt_summarize(consult: str) -> str:
    """Enhanced GPT summary with error handling"""
    if not config.enable_gpt_summary:
        return "GPT summary disabled in configuration"

    client = initialize_openai_client()
    if not client:
        return "GPT client initialization failed"

    try:
        prompt = f"""
You are a clinical AI engine trained to summarize neurological consults.

Summarize the following consult note and its flagged neurological findings:

Consult:
{consult}

Output a clear, 2–3 sentence summary with emphasis on the neuro exam and clinical concern.
Focus on critical findings, motor/sensory deficits, and risk factors.
"""
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a clinical summarizer for neurosurgical triage."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=200,
            temperature=0.3
        )
        return response.choices[0].message.content.strip()

    except Exception as e:
        clinical_logger.log_error(
            component="gpt_summary",
            error_code="SUMMARY_ERROR",
            error_message=str(e),
            severity="low"
        )
        return f"GPT summary failed: {str(e)}"

# === Enhanced CLI Interface ===
def format_findings_output(findings: ClinicalFindings) -> str:
    """Format clinical findings for display"""
    output_lines = []

    # Add critical findings first
    if findings.critical_flags:
        output_lines.append("🚨 CRITICAL FINDINGS:")
        for finding in findings.critical_flags:
            output_lines.append(f"  {finding.severity.value} {finding.message} (confidence: {finding.confidence:.2f})")
        output_lines.append("")

    # Warning findings
    if findings.warning_flags:
        output_lines.append("⚠️ WARNING FINDINGS:")
        for finding in findings.warning_flags:
            output_lines.append(f"  {finding.severity.value} {finding.message} (confidence: {finding.confidence:.2f})")
        output_lines.append("")

    # Motor findings
    if findings.motor_flags:
        output_lines.append("🦴 MOTOR FINDINGS:")
        for finding in findings.motor_flags:
            output_lines.append(f"  {finding.severity.value} {finding.message} (confidence: {finding.confidence:.2f})")
        output_lines.append("")

    # Neuro findings
    if findings.neuro_flags:
        output_lines.append("🧠 NEUROLOGICAL FINDINGS:")
        for finding in findings.neuro_flags:
            output_lines.append(f"  {finding.severity.value} {finding.message} (confidence: {finding.confidence:.2f})")
        output_lines.append("")

    # Monitoring findings
    if findings.monitoring_flags:
        output_lines.append("🔍 MONITORING:")
        for finding in findings.monitoring_flags:
            output_lines.append(f"  {finding.severity.value} {finding.message} (confidence: {finding.confidence:.2f})")
        output_lines.append("")

    # Normal findings
    if findings.normal_findings:
        output_lines.append("✅ NORMAL FINDINGS:")
        for finding in findings.normal_findings:
            output_lines.append(f"  {finding.severity.value} {finding.message} (confidence: {finding.confidence:.2f})")
        output_lines.append("")

    # Summary statistics
    total_findings = len(findings.get_all_findings())
    output_lines.append(f"📊 SUMMARY: {total_findings} findings, overall confidence: {findings.overall_confidence:.2f}, processing time: {findings.processing_time_ms}ms")

    return "\n".join(output_lines)

# === Synapse Clinical Analysis Interface ===
def run_synapse():
    """
    Main Synapse interface - allows users to paste clinical text and get analysis
    """
    print("🧠 SYNAPSE v2.0 — Clinical Risk Flagging Engine")
    print("=" * 60)
    print("Paste your clinical note below and press Enter to analyze:")
    print()

    # Get clinical text from user
    clinical_text = input("📋 Clinical Note: ")

    if not clinical_text.strip():
        print("❌ No text provided. Please paste a clinical note.")
        return

    print("\n🔍 Analyzing clinical findings...")
    print("=" * 60)

    try:
        # Process the clinical text
        findings = risk_flag(clinical_text)

        # Display results
        result = format_findings_output(findings)
        print(result)

        # Add GPT summary if enabled
        if config.enable_gpt_summary:
            print("\n📋 GPT CLINICAL SUMMARY:")
            print("-" * 30)
            summary = gpt_summarize(clinical_text)
            print(summary)

        print("\n" + "=" * 60)
        print("Analysis complete. Run again to analyze another case.")

    except Exception as e:
        print(f"❌ Error during analysis: {str(e)}")
        print("Please check your input and try again.")

# === Auto-run Synapse ===
def start_synapse():
    """
    Auto-start Synapse interface
    """
    while True:
        try:
            run_synapse()

            # Ask if user wants to analyze another case
            print("\nAnalyze another case? (y/n): ", end="")
            continue_analysis = input().lower().strip()

            if continue_analysis not in ['y', 'yes']:
                print("👋 Exiting Synapse. Thank you!")
                break

        except KeyboardInterrupt:
            print("\n👋 Exiting Synapse. Thank you!")
            break
        except Exception as e:
            print(f"❌ Unexpected error: {str(e)}")
            break

# === Entry Point ===
if __name__ == "__main__":
    # Check if running in Jupyter/Colab
    try:
        __IPYTHON__
        print("🧠 Synapse v2.0 loaded successfully!")
        print("\n" + "=" * 60)
        print("USAGE:")
        print("Just paste your clinical text below and press Enter:")
        print()

        # Direct input in Jupyter
        clinical_text = input("📋 Clinical Note: ")

        if clinical_text.strip():
            print("\n🔍 Analyzing clinical findings...")
            print("=" * 60)

            findings = risk_flag(clinical_text)
            result = format_findings_output(findings)
            print(result)

            if config.enable_gpt_summary:
                print("\n📋 GPT CLINICAL SUMMARY:")
                print("-" * 30)
                summary = gpt_summarize(clinical_text)
                print(summary)
        else:
            print("❌ No text provided. Please run again and paste a clinical note.")

    except NameError:
        # Command line mode
        start_synapse()

🧠 Synapse v2.0 loaded successfully!

USAGE:
Just paste your clinical text below and press Enter:



KeyboardInterrupt: Interrupted by user